First, run the instructions in here - [Accessing AWS](https://www.notion.so/lattice/Accessing-AWS-043c558c833044e38329ef3dc9c1ea26#4e7d707777864670814e37aef0a08a6b) which will log you in using your okta roles. this will last 4 hours.

In [5]:
import boto3
import csv
import json
from botocore.exceptions import NoCredentialsError, PartialCredentialsError

def flatten_json(y, parent_key='', sep='.'):
    """
    Flattens a nested JSON object.

    :param y: The JSON object to flatten.
    :param parent_key: The base key string.
    :param sep: Separator between keys.
    :return: A flattened dictionary.
    """
    items = {}
    for k, v in y.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(flatten_json(v, new_key, sep=sep))
        elif isinstance(v, list):
            # Optionally, handle lists by joining items with a separator or indexing
            items[new_key] = '; '.join(map(str, v))
        else:
            items[new_key] = v
    return items

def main():
    try:
        # Initialize a session using the default credentials (managed by ktool)
        session = boto3.Session(
            region_name='us-west-2' , # Replace with your AWS region
            profile_name='lattice'
        )

        print(session.available_profiles)

        # Create SQS client using the session
        sqs = session.client('sqs')

        # Get the URL for the SQS queue
        queue_url = sqs.get_queue_url(QueueName='weaver-worker-text-analysis-dlq')['QueueUrl']

        # Receive messages from SQS queue
        response = sqs.receive_message(
            QueueUrl=queue_url,
            # MaxNumberOfMessages=10,  # Adjust the number of messages to read
            VisibilityTimeout=10,
            WaitTimeSeconds=10
        )

        messages = response.get('Messages', [])

        if not messages:
            print("No messages available in the queue.")
            return

        # Prepare to collect all unique keys for CSV headers
        csv_headers = set(['MessageId', 'ReceiptHandle'])  # Initial headers

        # List to hold all message data
        all_messages = []

        for message in messages:
            message_id = message.get('MessageId', '')
            receipt_handle = message.get('ReceiptHandle', '')
            body = message.get('Body', '{}')  # Default to empty JSON if Body is missing

            try:
                # Parse the JSON body
                body_json = json.loads(body)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON for MessageId {message_id}: {e}")
                body_json = {}

            # Flatten the JSON body
            flattened_body = flatten_json(body_json)

            # Update headers with keys from the flattened JSON
            csv_headers.update(flattened_body.keys())

            # Combine message metadata with flattened body
            combined_message = {
                'MessageId': message_id,
                'ReceiptHandle': receipt_handle
            }
            combined_message.update(flattened_body)

            all_messages.append(combined_message)

        # Define the order of CSV headers (optional)
        ordered_headers = ['MessageId', 'ReceiptHandle'] + sorted(k for k in csv_headers if k not in ['MessageId', 'ReceiptHandle'])

        # Write CSV data to a file
        with open('sqs_messages.csv', 'w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=ordered_headers)
            writer.writeheader()
            for msg in all_messages:
                writer.writerow(msg)

        print(f"{len(all_messages)} messages have been written to sqs_messages.csv")

    except NoCredentialsError:
        print("Error: AWS credentials not found. Please authenticate using ktool.")
    except PartialCredentialsError:
        print("Error: Incomplete AWS credentials. Please check your ktool configuration.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    main()

['lattice-local', 'default', 'lattice']
10 messages have been written to sqs_messages.csv
